In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yt_dlp 
import librosa 
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import data_processor
import weaviate_database 
from torch.utils.data import DataLoader
import weaviate 
import torch 
import llama_chat
from typing import List 
import download_audio 
# https://www.youtube.com/watch?v=XkxZCJ1pYqs

In [3]:
def process_video(url, file): 
  data, sr = download_audio.main(url, f'audio_files/{file}.m4a')
  assert sr == 16000 

  data_processor.speech_recognition(data, f'data/{file}.txt')

  with open(f'data/{file}.txt', 'r') as file: 
    prediction = file.read()

  all_text = data_processor.slice_text(prediction)
  text_dl = DataLoader(all_text, batch_size=128, shuffle=False) 
  return text_dl 

In [4]:
client = weaviate.Client(url='http://localhost:8080', timeout_config=600)
client.batch.batch_size = 1
[print(x['class']) for x in client.schema.get()['classes']]

Huberman
YT


[None, None]

In [5]:
text_dl = process_video('https://www.youtube.com/watch?v=TnB_8Zm9lPk', 'Aus')

[youtube] Extracting URL: https://www.youtube.com/watch?v=TnB_8Zm9lPk
[youtube] TnB_8Zm9lPk: Downloading webpage
[youtube] TnB_8Zm9lPk: Downloading ios player API JSON
[youtube] TnB_8Zm9lPk: Downloading android player API JSON
[youtube] TnB_8Zm9lPk: Downloading m3u8 information
[info] TnB_8Zm9lPk: Downloading 1 format(s): 251
[download] Destination: audio_files/Aus.m4a
[download] 100% of   30.14MiB in 00:00:24 at 1.23MiB/s   


/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/audioread/rawread.py:16: DeprecationWarning: 'aifc' is deprecated and slated for removal in Python 3.13
  import aifc
/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/audioread/rawread.py:17: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/audioread/rawread.py:19: DeprecationWarning: 'sunau' is deprecated and slated for removal in Python 3.13
  import sunau
/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/audioread/ffdec.py:114: ResourceWarning: unclosed file <_io.BufferedReader name=80>
  return proc.returncode == 0
/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/audioread/ffdec.py:114: ResourceWarning: unclosed file <_io.BufferedReader name=82>
  return proc.returncode == 0
/mnt/sda1/Programming/yt_llama_qa/download_audio.py:11: UserWarning: PySoundFile

In [10]:
weaviate_database.add_data_to_db(client, 'Aus', text_dl, new_db=True)

Iterations: [1/1]: 100%|██████████| 9/9 [00:00<00:00, 186.42it/s]


In [11]:
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('/mnt/sda1/hf_models/Llama-2-7b-chat-hf/')
model = AutoModelForCausalLM.from_pretrained('/mnt/sda1/hf_models/Llama-2-7b-chat-hf/', device_map=device, torch_dtype=torch.float16,  use_cache=True) 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
query = 'Why is Australia so empty?'
results = weaviate_database.search_db(client, 'Aus', query)

prompt = f'''<<SYS>>
You are a question answering bot that answers questions by reading video transcripts provided by the user. 
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If the answer is not contained in the transcript given, please don't share false information.
Give short succinct answer that uses key words from the transcript. 
<</SYS>>

[INST] {llama_chat.get_prompt(results, tokenizer).strip()}

Based on the video transcript above answer the following question. 
Question: {query}
Answer:[/INST]'''

print(llama_chat.ask_llama(prompt, tokenizer, model, device))

/mnt/sda1/python_environments/AI3_11/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


<s> <<SYS>>
You are a question answering bot that answers questions by reading video transcripts provided by the user. 
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If the answer is not contained in the transcript given, please don't share false information.
Give short succinct answer that uses key words from the transcript. 
<</SYS>>

[INST] * This dot is the city of Adelaide, Australia's fifth largest city home to more than 1.3 million people. And immediately adjacent to this oasis of urban life is the pastoral unincorporated area, a territory that's roughly the size of France, but is only home to a whopping 3,750 people, making the population density something like 178 square kilometers of land for every one person. That is roughly one Aruba's worth of land for every single resident inside of the pastoral unincorporated area. Then there's also Anna Creek right over here. This is a huge territory that's s